## Simulação de portfólios e indicação de carteiras ótimas

#### BLOCO 1 - Definição dos ativos

In [20]:
# ====== Configurações ======
DATA_INICIO = "2020-01-01"
DATA_FIM = "2026-01-01"
DIAS_UTEIS_ANO = 252

# ====== Ativos (moeda-base: BRL) ======
# Brasil (B3) - já em BRL
ativos_br = {
    "PETR4": "PETR4.SA",
    "ITUB4": "ITUB4.SA",
    "VALE3": "VALE3.SA",
    "BBAS3": "BBAS3.SA",
    "QBTC11": "QBTC11.SA",
}

# EUA - em USD (serão convertidos para BRL)
ativos_us = {
    "GOOGL": "GOOGL",
    "NVDA": "NVDA",
    "NDAQ": "NDAQ",
    "META": "META",
    "AMZN": "AMZN",
    "VOO": "VOO",
}

# Cripto - em USD (serão convertidos para BRL)
cripto_usd = {
    "BTC": "BTC-USD",
    "SOL": "SOL-USD",
}

# USDC = CAIXA (fora da otimização)
# Se quiser usar na carteira final como % em caixa, defina aqui:
PESO_CAIXA_USDC = 0.00  # ex.: 0.10 para 10% em caixa (USDC)
RETORNO_CAIXA_ANUAL = 0.0  # opcional: defina um retorno anual pro caixa (ex.: CDI aproximado)


#### Bloco 2 - Bibliotecas Necessárias

In [21]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
# import matplotlib.pyplot as plt
import plotly.graph_objects as go

#### Etapa 1: Download + conversão USD→BRL + retornos e cov (substitua o seu bloco “Baixar dados / Calcular retornos”)

In [22]:

def _baixar_preco_adjclose(tickers, start, end):
    raw = yf.download(tickers, start=start, end=end, progress=False, auto_adjust=False)
    if raw.empty:
        return pd.DataFrame()

    # Preferir Adj Close, mas fazer fallback para Close
    if isinstance(raw.columns, pd.MultiIndex):
        if "Adj Close" in raw.columns.get_level_values(0):
            px = raw["Adj Close"].copy()
        elif "Close" in raw.columns.get_level_values(0):
            px = raw["Close"].copy()
        else:
            raise ValueError("Não encontrei colunas 'Adj Close' nem 'Close' no retorno do yfinance.")
    else:
        # Caso raro: sem MultiIndex
        px = raw.copy()

    px = px.dropna(axis=1, how="all")
    return px

def _baixar_usdbrl(start, end):
    # Tenta os dois formatos mais comuns no Yahoo
    for fx_ticker in ["BRL=X", "USDBRL=X"]:
        fx = _baixar_preco_adjclose([fx_ticker], start, end)
        if not fx.empty:
            s = fx.iloc[:, 0].rename("USDBRL").dropna()
            return s
    raise ValueError("Não consegui baixar o câmbio USD/BRL (tentei 'BRL=X' e 'USDBRL=X').")

# ====== Monta lista de tickers a baixar (USDC fora) ======
tickers_br = list(ativos_br.values())
tickers_us = list(ativos_us.values())
tickers_crypto = list(cripto_usd.values())

TICKERS = tickers_br + tickers_us + tickers_crypto

# ====== Baixar preços ======
precos = _baixar_preco_adjclose(TICKERS, DATA_INICIO, DATA_FIM)
if precos.empty:
    raise ValueError("Download retornou vazio. Verifique tickers, conexão e período.")

# ====== Baixar câmbio USD/BRL e alinhar datas ======
usdbrl = _baixar_usdbrl(DATA_INICIO, DATA_FIM)

# Alinha tudo no mesmo calendário (interseção)
df = precos.join(usdbrl, how="inner")

if df.empty:
    raise ValueError("Após alinhar com o câmbio, não restaram datas em comum.")

# ====== Converter colunas USD -> BRL ======
usd_cols = [c for c in (tickers_us + tickers_crypto) if c in df.columns]
for c in usd_cols:
    df[c] = df[c] * df["USDBRL"]

# Remove coluna do câmbio do dataframe de preços
dados_brl = df.drop(columns=["USDBRL"])

# ====== Renomear colunas para os nomes “lógicos” (PETR4, NVDA, BTC, etc.) ======
# Cria mapa ticker->nome
ticker_to_nome = {v: k for k, v in ativos_br.items()}
ticker_to_nome.update({v: k for k, v in ativos_us.items()})
ticker_to_nome.update({v: k for k, v in cripto_usd.items()})

dados_brl = dados_brl.rename(columns=ticker_to_nome)

# Remove colunas eventualmente faltantes
dados_brl = dados_brl.dropna(axis=1, how="all")
if dados_brl.shape[1] < 2:
    raise ValueError("Poucos ativos com dados válidos após limpeza. Verifique tickers.")

# ====== Retornos ======
retornos = dados_brl.pct_change().dropna()

# ====== Estatísticas anualizadas (Markowitz) ======
retornos_medios_anuais = retornos.mean() * DIAS_UTEIS_ANO
matriz_covariancia = retornos.cov() * DIAS_UTEIS_ANO

print("Ativos usados na otimização (tudo em BRL):", list(retornos.columns))
print("PESO_CAIXA_USDC (fora da otimização):", PESO_CAIXA_USDC)


Ativos usados na otimização (tudo em BRL): ['AMZN', 'BBAS3', 'BTC', 'GOOGL', 'ITUB4', 'META', 'NDAQ', 'NVDA', 'PETR4', 'QBTC11', 'SOL', 'VALE3', 'VOO']
PESO_CAIXA_USDC (fora da otimização): 0.0


C:\Users\ERASMO GUSMAO\AppData\Local\Temp\ipykernel_3600\4140501404.py:73: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



#### Etapa 2: Funções de Cálculo das Métricas

In [23]:
def calcular_metricas_portfolio(pesos, retornos_medios_anuais, matriz_covariancia):
    """
    Calcula retorno, volatilidade e Sharpe Ratio de um portfólio.
    
    Parâmetros:
        pesos: array numpy com os pesos de cada ativo
        retornos_medios_anuais: série pandas com retornos esperados
        matriz_covariancia: matriz de covariância dos retornos
    
    Retorna:
        array [retorno, volatilidade, sharpe_ratio]
    """
    pesos = np.array(pesos)
    
    # Retorno do portfólio: Rp = Σ(Wi × Ri)
    retorno_portfolio = np.sum(retornos_medios_anuais * pesos)
    
    # Volatilidade: σp = √(W^T × Σ × W)
    volatilidade_portfolio = np.sqrt(np.dot(pesos.T, np.dot(matriz_covariancia, pesos)))
    
    # Sharpe Ratio (assumindo taxa livre de risco = 0) <- MELHORIA usar dado real do CDI
    sharpe_ratio = retorno_portfolio / volatilidade_portfolio
    
    return np.array([retorno_portfolio, volatilidade_portfolio, sharpe_ratio])


#### Etapa 3: Funções Objetivo para Otimização

In [24]:
def minimizar_volatilidade(pesos, retornos_medios_anuais, matriz_covariancia):
    """
    Função objetivo para encontrar o portfólio de Mínima Variância.
    Retorna apenas a volatilidade (índice 1).
    """
    return calcular_metricas_portfolio(pesos, retornos_medios_anuais, matriz_covariancia)[1]

def maximizar_sharpe(pesos, retornos_medios_anuais, matriz_covariancia):
    """
    Função objetivo para encontrar o portfólio de Sharpe Máximo.
    Como minimize() busca o mínimo, retornamos o Sharpe negativo.
    """
    return -calcular_metricas_portfolio(pesos, retornos_medios_anuais, matriz_covariancia)[2]


#### Etapa 4: Configuração da Otimização

In [25]:
# Parâmetros
num_ativos = len(TICKERS)
pesos_iniciais = np.array([1.0/num_ativos] * num_ativos)  # Distribuição igual

# Restrições
limites = tuple((0, 1) for _ in range(num_ativos))  # Cada peso entre 0% e 100%
restricoes = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Soma = 1

# Otimização para Carteira de Mínima Variância
otimizacao_min_vol = minimize(
    minimizar_volatilidade,
    pesos_iniciais,
    args=(retornos_medios_anuais, matriz_covariancia),
    method='SLSQP',  # Sequential Least Squares Programming
    bounds=limites,
    constraints=restricoes
)

pesos_min_vol = otimizacao_min_vol.x
metricas_min_vol = calcular_metricas_portfolio(pesos_min_vol, retornos_medios_anuais, matriz_covariancia)

# Otimização para Portfólio de Sharpe Máximo
otimizacao_max_sharpe = minimize(
    maximizar_sharpe,
    pesos_iniciais,
    args=(retornos_medios_anuais, matriz_covariancia),
    method='SLSQP',
    bounds=limites,
    constraints=restricoes
)

pesos_max_sharpe = otimizacao_max_sharpe.x
metricas_max_sharpe = calcular_metricas_portfolio(pesos_max_sharpe, retornos_medios_anuais, matriz_covariancia)


#### Etapa 5: Simulação de Monte Carlo para Visualizar a Fronteira

In [26]:
# Simulação de 100.000 portfólios aleatórios
num_portfolios = 100000
resultados = np.zeros((3, num_portfolios))  # [retorno, risco, sharpe]

np.random.seed(42)  # Para reprodutibilidade

for i in range(num_portfolios):
    # Gerar pesos aleatórios
    pesos_aleatorios = np.random.random(num_ativos)
    pesos_aleatorios /= np.sum(pesos_aleatorios)  # Normalizar para somar 1
    
    # Calcular métricas
    metricas = calcular_metricas_portfolio(pesos_aleatorios, retornos_medios_anuais, matriz_covariancia)
    resultados[0, i] = metricas[0]  # retorno
    resultados[1, i] = metricas[1]  # risco
    resultados[2, i] = metricas[2]  # sharpe


#### Etapa 6: Visualização da Fronteira Eficiente

In [27]:
# Visualização interativa (Plotly) - mantendo o padrão de cores (Viridis + pontos destaque vermelho/verde)

fig = go.Figure()

# Nuvem de portfólios aleatórios (coloridos pelo Sharpe Ratio)
fig.add_trace(
    go.Scatter(
        x=resultados[1, :],  # Volatilidade (Risco) - %
        y=resultados[0, :],  # Retorno Esperado - %
        mode="markers",
        name="Portfólios (Monte Carlo)",
        marker=dict(
            color=resultados[2, :],
            colorscale="Viridis",     # mesmo padrão do 'cmap=viridis'
            showscale=True,
            colorbar=dict(title="Índice de Sharpe"),
            size=6,
            opacity=0.5,
            symbol="circle",
        ),
        hovertemplate=(
            "Volatilidade: %{x:.2f}%<br>"
            "Retorno: %{y:.2f}%<br>"
            "Sharpe: %{marker.color:.3f}"
            "<extra></extra>"
        ),
    )
)

# Destacar Carteira de Mínima Variância
fig.add_trace(
    go.Scatter(
        x=[metricas_min_vol[1]],
        y=[metricas_min_vol[0]],
        mode="markers",
        name="Mínima Volatilidade",
        marker=dict(
            symbol="star",
            size=18,
            color="red",
            line=dict(color="black", width=2),
        ),
        hovertemplate=(
            "<b>Mínima Volatilidade</b><br>"
            "Volatilidade: %{x:.2f}%<br>"
            "Retorno: %{y:.2f}%"
            "<extra></extra>"
        ),
    )
)

# Destacar Portfólio de Sharpe Máximo
fig.add_trace(
    go.Scatter(
        x=[metricas_max_sharpe[1]],
        y=[metricas_max_sharpe[0]],
        mode="markers",
        name="Sharpe Máximo",
        marker=dict(
            symbol="star",
            size=18,
            color="green",
            line=dict(color="black", width=2),
        ),
        hovertemplate=(
            "<b>Sharpe Máximo</b><br>"
            "Volatilidade: %{x:.2f}%<br>"
            "Retorno: %{y:.2f}%"
            "<extra></extra>"
        ),
    )
)

# Formatação (equivalente ao Matplotlib)
fig.update_layout(
    title=dict(text="Fronteira Eficiente de Markowitz", x=0.5),
    xaxis_title="Volatilidade Anualizada (Risco) - %",
    yaxis_title="Retorno Esperado Anualizado - %",
    template="plotly_white",
    legend=dict(x=0.01, y=0.99),
    width=950,
    height=650,
)

# Grade leve (similar ao plt.grid(alpha=0.3))
fig.update_xaxes(showgrid=True, gridwidth=1)
fig.update_yaxes(showgrid=True, gridwidth=1)

fig.show()


#### Etapa 7: Visual da matriz de covariância (Plotly Heatmap: frio/quente)

In [28]:
# Visualização da Matriz de Covariância (Plotly) - cores frias/quentes conforme sinal e magnitude
cov = matriz_covariancia.copy()

fig_cov = go.Figure(
    data=go.Heatmap(
        z=cov.values,
        x=list(cov.columns),
        y=list(cov.index),
        colorscale="RdBu",   # azul (negativo) ↔ vermelho (positivo)
        zmid=0,              # centraliza em 0 (essencial p/ quente/frio)
        colorbar=dict(title="Covariância<br>(%²/ano)"),
        hovertemplate="Ativo X: %{x}<br>Ativo Y: %{y}<br>Cov: %{z:.4f}<extra></extra>",
    )
)

fig_cov.update_layout(
    title=dict(text="Matriz de Covariância Anualizada", x=0.5),
    template="plotly_white",
    width=750,
    height=650,
)

fig_cov.update_xaxes(side="top")
fig_cov.show()
